## Splitting Continuous attribute

- 연속적인 값을 가진 attribute를 기준으로 branch를 나누기

### Continuous Attribute 나누기

- 불연속적 명목 데이터에 비해 나눌 수 있는 구간이 많음
1. 전체 데이터를 모두 기준점으로 한다
2. 중위값, 4분위수들을 기준점으로 한다(binning)
3. y-class 값이 바뀌는 수를 기준점으로 한다

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd_data = pd.read_csv(
    './vegeterianl_dataset.csv',  delimiter=r"\s+")
pd_data.drop("ID",axis=1)
pd_data

,ID,STREAM,SLOPE,ELEVATION,VEGETATION
0,1,False,steep,3900,chapparal
1,2,True,moderate,300,riparian
2,3,True,steep,1500,riparian
3,4,False,steep,1200,chapparal
4,5,False,flat,4450,conifer
5,6,True,steep,5000,conifer
6,7,True,steep,3000,chapparal


In [4]:
# Sorting을 먼저 해준다
pd_data.sort_values("ELEVATION")

,ID,STREAM,SLOPE,ELEVATION,VEGETATION
1,2,True,moderate,300,riparian
3,4,False,steep,1200,chapparal
2,3,True,steep,1500,riparian
6,7,True,steep,3000,chapparal
0,1,False,steep,3900,chapparal
4,5,False,flat,4450,conifer
5,6,True,steep,5000,conifer


In [5]:
# Entropy 구하기
def get_info(df):
    riparian = df.loc[df["VEGETATION"]=="riparian"]
    chapparal = df.loc[df["VEGETATION"]=="chapparal"]
    conifer = df.loc[df["VEGETATION"]=="conifer"]

    x = np.array([len(riparian)/len(df),len(chapparal)/len(df), 
                  len(conifer)/len(df)])
    y = np.log2(x[x!=0])
    

    info_all = - sum(x[x!=0] * y)
    return info_all

In [6]:
get_info(pd_data)

1.5566567074628228

In [7]:
def get_attribute_info(df, attribute_name, continuous_value=0):
    get_infos = []
    if continuous_value == 0:
        attribute_values = pd_data[attribute_name].unique()
        for value in attribute_values:
            split_df = pd_data.loc[pd_data[attribute_name] == value]
            get_infos.append((len(split_df) / len(df)) * get_info(split_df))
    else:
        split_df_1 = pd_data.loc[pd_data[attribute_name] >= continuous_value]
        split_df_2 = pd_data.loc[pd_data[attribute_name] < continuous_value]

        get_infos.append((len(split_df_1) / len(df)) * get_info(split_df_1))                     
        get_infos.append((len(split_df_2) / len(df)) * get_info(split_df_2))                     
    return sum(get_infos)

In [10]:
# split index를 구해서 두 값 사이의 평균을 구함
def get_continuos_attribute_info(df, attribute_name):
    result = pd_data.sort_values(attribute_name)
    prior = 0 
    split_index = []
    sorted_key = {}
    sorted_index = {}
    for index, value in enumerate(result.index):
        if prior != result["VEGETATION"][value]:  # 이전의 ylabel과 현재의 ylabel이 다른 경우에만 split index로 기록
            split_index.append(value)
            prior = result["VEGETATION"][value]
        sorted_index[value] = index
        sorted_key[index] = value
    split_value = []
    for v in split_index[1:]:
        current_value = result[attribute_name][v]
        prior_value_index = sorted_key[sorted_index[v] - 1]
        prior_value = result[attribute_name][prior_value_index]
        
        split_value_average = current_value + prior_value
        split_value.append(split_value_average/2)
    result = {}
    for value in split_value:
        result[value] = (get_attribute_info(df, attribute_name, value))
    return result

In [13]:
# 4175가 information gain이 가장 높음. 따라서 4715를 기준으로 branch를 나누는 것이 좋음
get_continuos_attribute_info(pd_data, "ELEVATION")

{750.0: 1.250698214594781,
 1350.0: 1.3728057820624016,
 2250.0: 0.9649839288804954,
 4175.0: 0.6935361388961918}

### Continuous attribute branch 특징

- 명목속성과 달리, 여러번 재사용 가능. 단, 경계값은 달라야 함
- 연속값과 명목값을 동시에 split 가능